# Проект по SQL

Цель исследования: проанализировать базу данных

Данные: информация о книгах, издательствах, авторах, а также пользовательские обзоры книг

## Подключение к базе данных

Импортируем библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Зададим параметры подключения к базе данных

In [2]:
db_config = {
    'user': 'praktikum_student',  # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',  # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',  # адрес сервера
    'port': 6432,  # порт подключения
    'db': 'data-analyst-final-project-db',  # название базы данных}
}

Создадим подключение

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследуем таблицы

In [5]:
query = '''
SELECT *
FROM books
'''
books = pd.io.sql.read_sql(query, con=engine)
display(books.head(5))
books.info()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [6]:
query1 = '''
SELECT *
FROM authors
'''
authors = pd.io.sql.read_sql(query1, con=engine)
display(authors.head(5))
authors.info()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [7]:
query2 = '''
SELECT *
FROM ratings
'''
ratings = pd.io.sql.read_sql(query2, con=engine)
display(ratings.head(5))
ratings.info()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [8]:
query3 = '''
SELECT *
FROM reviews
'''
reviews = pd.io.sql.read_sql(query3, con=engine)
display(reviews.head(5))
reviews.info()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [9]:
query4 = '''
SELECT *
FROM publishers
'''
publishers = pd.io.sql.read_sql(query4, con=engine)
display(publishers.head(5))
publishers.info()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


Заметим, что все данные в верном типе.

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [10]:
query_1 = '''
SELECT count(book_id) as Количество
FROM books
where publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query_1, con=engine)

,Количество
0,819


После 1 января 2000 года было выпущено 819 книг

## Для каждой книги посчитаем количество обзоров и среднюю оценку и выведем книги с наибольшим количеством обзоров

In [11]:
query_2 = '''
select title, count_reviews, round(avg(rating), 2) as avg_rating
from ratings
    right join (
        SELECT book_id, title, count(review_id) as count_reviews
        FROM books
            left join reviews using (book_id)
        group by book_id, title) as q1 using(book_id)
group by title, count_reviews
order by count_reviews desc
limit 10
'''
pd.io.sql.read_sql(query_2, con=engine)

,title,count_reviews,avg_rating
0,Twilight (Twilight #1),7,3.66
1,The Road,6,3.77
2,The Curious Incident of the Dog in the Night-Time,6,4.08
3,The Alchemist,6,3.79
4,Outlander (Outlander #1),6,4.13
5,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
6,The Book Thief,6,4.26
7,The Catcher in the Rye,6,3.83
8,The Glass Castle,6,4.21
9,The Hobbit or There and Back Again,6,4.13


Больше всего обзоров у книги Twilight - 7, далее идут The Road, The Curious Incident of the Dog in the Night-Time, The Alchemist и другие с 6 обзорами 

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [12]:
query_3 = '''
select publisher, count(book_id) as count_book
from books
    inner join publishers using (publisher_id)
group by publisher
having count(book_id)  = (
            select count(book_id) as cnt
            from books
                inner join publishers using (publisher_id)
            where num_pages > 50
            group by publisher
            order by cnt desc
            limit 1
            )
'''
pd.io.sql.read_sql(query_3, con=engine)

,publisher,count_book
0,Penguin Books,42


Больше всего книг выпустило издательство Penguin Books

## Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками

In [13]:
query_4 = '''
select author, round(avg(rating), 2) as avg_rating
from books
    inner join ratings using (book_id)
    inner join authors using (author_id)
where book_id in (
            select book_id
            from books
                inner join ratings using (book_id)
            group by book_id
            having count(rating_id) >= 50
            )
group by author
order by avg(rating) desc
'''
pd.io.sql.read_sql(query_4, con=engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
9,Lois Lowry,3.75


Самая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
query_5 = '''
select round(avg(cnt), 2) as Среднее
from (
            select username, count(review_id) as cnt
            from reviews
                inner join (
                        select username
                        from ratings
                        group by username
                        having count(rating) > 50) as q1 using (username)
            group by username) as q2
'''
pd.io.sql.read_sql(query_5, con=engine)

,Среднее
0,24.33


У пользователей, которые поставили больше 50 оценок среднее количество обзоров равняется 24.33

## Вывод

После 1 января 2000 года было выпущено 819 книг. Больше всего обзоров с большим отрывом у книги Twilight - 1120, далее идут The Hobbit or There and Back Again и The Catcher in the Rye с чуть более 500 обзорами. Больше всего книг выпустило издательство Penguin Books. Самая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré. У пользователей, которые поставили больше 50 оценок среднее количество обзоров равняется 24.33